# from before:

In [1]:
import pandas as pd
import numpy as np
import bqplot
import ipywidgets
# our usual stuff
%matplotlib inline
# lets add in some exprot data
comm = pd.read_csv('/Users/jillnaiman/Downloads/total_export.csv')

In [2]:
# grab years
years = list(comm.columns.values)
years = np.array(years[1:]) # get rid of state
# as numbers
years = years.astype('int')
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [3]:
sc_geo = bqplot.AlbersUSA()
state_data = bqplot.topo_load('map_data/USStatesMap.json')

def_tt = bqplot.Tooltip(fields=['id', 'name'])

states_map = bqplot.Map(map_data=state_data, scales={'projection':sc_geo}, tooltip=def_tt)
states_map.interactions = {'click': 'select', 'hover': 'tooltip'}

fig=bqplot.Figure(marks=[states_map], title='US States Map Example',
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})


# lets also make a line plot
# second, the lineplot
x_scl = bqplot.LinearScale() 
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Year', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total Export from State NA', 
                     scale=y_scl, 
                    orientation='vertical', side='left')
lines = bqplot.Lines(x = years, y = np.zeros(len(years)),
                    scales = {'x': x_scl, 'y': y_scl})
#print(lines)
fig_lines = bqplot.Figure(marks = [lines], 
                          axes = [ax_ycl, ax_xcl],)

# let do something additive for all states selected
def get_data_value(change):
    exports = np.zeros(len(years))
    snames = ''
    if change['owner'].selected is not None:
        for i,s in enumerate(change['owner'].selected):
            sn = state_names[s == ids][0]
            snames += sn + ', '
            # because of formatting, things are in arrays hence [0]
            #  also, take out state name hence [1:]
            # NOTE! BQPLOT has misspelled massachussetts!
            if sn == 'Massachusetts': sn = 'Massachussetts'
            exports_in=comm.loc[comm['State'] == sn].values[0][1:]
            # there are ","'s in exports we gotta take out
            exports_in = np.array([exports_in[i].replace(',','') for i in range(len(exports_in))])
            exports = np.add(exports, exports_in.astype('float64'))
        lines.y = exports
        ax_ycl.label='Total Export from ' + snames
    else:
        lines.y = np.zeros(len(exports))
        ax_ycl.label='Total Export from NA'

states_map.observe(get_data_value,'selected')

# some formatting for vertical
#fig_lines.layout.max_height='250px'
#fig_lines.layout.min_width='800px'
#fig.layout.min_width='800px'
#ipywidgets.VBox([fig_lines,fig])
ipywidgets.HBox([fig,fig_lines])

# Activity #1: MarketMap
* another way to visualize mappable data

## 1.a : explore the dataset

In [4]:
#!pip install xlrd # JPN, might have to run this

# note: this is quering from the web!  How neat is that??
df = pd.read_excel('https://query.data.world/s/ivl45pdpubos6jpsii3djsjwm2pcjv', skiprows=5)
# the above might take a while to load all the data

In [5]:
# save as CSV if you wanna
#df.to_csv('/Users/jillnaiman/Downloads/market_map_data.csv')

In [6]:
# or can try:
#df = pd.read_csv('/Users/jillnaiman/Downloads/market_map_data.csv')

In [7]:
# what is in this dataframe? lets take a look at the top
df.head()
# this dataset is called: "Surgery Charges Across the U.S."
#  and its just showing us how much different procedures 
#  cost from different hospitals

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,10033,UNIVERSITY OF ALABAMA HOSPITAL,619 SOUTH 19TH STREET,BIRMINGHAM,AL,35233,AL - Birmingham,13,1.172866e+06,251876.307692,244457.923077
1,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,30103,MAYO CLINIC HOSPITAL,5777 EAST MAYO BOULEVARD,PHOENIX,AZ,85054,AZ - Phoenix,20,4.375313e+05,240422.800000,133509.550000
2,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50108,SUTTER GENERAL HOSPITAL,2801 L STREET,SACRAMENTO,CA,95816,CA - Sacramento,25,8.156741e+05,233197.480000,221681.800000
3,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50262,RONALD REAGAN U C L A MEDICAL CENTER,757 WESTWOOD PLAZA,LOS ANGELES,CA,90095,CA - Los Angeles,14,1.499044e+06,415968.785714,366608.928571
4,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,50441,STANFORD HOSPITAL,300 PASTEUR DRIVE,STANFORD,CA,94305,CA - San Mateo County,23,2.238699e+06,420865.478261,403453.652174


In [8]:
# what kinds of data are we working with?
df.dtypes

DRG Definition                                 object
Provider Id                                     int64
Provider Name                                  object
Provider Street Address                        object
Provider City                                  object
Provider State                                 object
Provider Zip Code                               int64
Hospital Referral Region (HRR) Description     object
Total Discharges                                int64
Average Covered Charges                       float64
Average Total Payments                        float64
Average Medicare Payments                     float64
dtype: object

In [9]:
# lets look at some summary data
# recall: this is like R's "summary" function
df.describe()
# so, things like the mean zipcode aren't
#  meaningful, same thing with provider ID
#  But certainly looking at the average
#  total payments, discharges, might 
#  be useful

,Provider Id,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
count,202656.000000,202656.000000,202656.000000,2.026560e+05,202656.000000,202656.000000
mean,255322.552749,47029.141698,36.011196,5.258934e+04,13168.560144,11218.465217
std,150834.203550,27792.935433,48.864835,6.244522e+04,13772.839671,12279.103622
min,10001.000000,1040.000000,11.000000,1.367652e+03,2320.482759,1329.909091
25%,110082.000000,25301.000000,14.000000,2.094930e+04,6258.925806,5011.979167
50%,240093.000000,43701.000000,22.000000,3.463380e+04,9180.169753,7714.654762
75%,380014.000000,71603.000000,39.000000,6.076193e+04,14631.466690,12472.680288
max,670088.000000,99801.000000,3855.000000,2.238699e+06,434396.000000,403453.652174


In [10]:
# lets look at how many seperate types of surgery are 
#  represented in this dataset:
df["DRG Definition"].unique().size

564

In [11]:
# what about how many provider (hospital) names?
df["Provider Name"].unique().size

3151

In [12]:
# how many states are represented
df["Provider State"].unique().size

51

In [13]:
# what are the state codes?
df["Provider State"].unique()

array(['AL', 'AZ', 'CA', 'CT', 'DC', 'FL', 'GA', 'IL', 'IN', 'KY', 'LA',
       'MA', 'MI', 'MN', 'MO', 'NE', 'NJ', 'NY', 'NC', 'OH', 'OK', 'PA',
       'SC', 'TN', 'TX', 'VA', 'WA', 'WI', 'AK', 'AR', 'CO', 'DE', 'ID',
       'IA', 'KS', 'ME', 'MD', 'MS', 'NV', 'NH', 'NM', 'ND', 'OR', 'RI',
       'UT', 'VT', 'WV', 'HI', 'SD', 'MT', 'WY'], dtype=object)

In [14]:
# lets figure out  what the most common surgeries are via how 
#  many many folks are discharged after each type of surgery
# (1)
most_common = df.groupby("DRG Definition")["Total Discharges"].sum()
most_common

# (2) but lets sort by the largest on top
most_common = df.groupby("DRG Definition")["Total Discharges"].sum().sort_values(ascending=False)
most_common

# (3) lets look at only the top 5, for fun
most_common[:5]

# (4) or we can only look at the names of the top 5:
most_common[:5].index.values

array(['470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC',
       '871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC',
       '291 - HEART FAILURE & SHOCK W MCC',
       '292 - HEART FAILURE & SHOCK W CC',
       '392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC'],
      dtype=object)

## 1.b: formatting data for MarketMap
* here we are going to practice doing some fancy things to clean this data
* this will be good practice for when you run into other datasets "in the wild"

In [15]:
# (1) lets create a little table of total discharges for
#  each type of surgery & state
total_discharges = df.groupby(["DRG Definition", "Provider State"])["Total Discharges"].sum()
total_discharges

# (2) the above is not intuative, lets prettify it
total_discharges = df.groupby(["DRG Definition", "Provider State"])["Total Discharges"].sum().unstack()
total_discharges

Provider State,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
DRG Definition,,,,,,,,,,,,,,,,,,,,,
001 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W MCC,NaN,13.0,NaN,20.0,125.0,NaN,29.0,31.0,NaN,83.0,...,NaN,34.0,206.0,NaN,61.0,NaN,36.0,21.0,NaN,NaN
002 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W/O MCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W MAJ O.R.",12.0,237.0,128.0,200.0,710.0,117.0,125.0,128.0,79.0,934.0,...,NaN,381.0,906.0,54.0,356.0,13.0,74.0,114.0,84.0,NaN
"004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W/O MAJ O.R.",11.0,257.0,129.0,127.0,894.0,59.0,59.0,81.0,55.0,943.0,...,18.0,438.0,1009.0,12.0,247.0,NaN,42.0,84.0,50.0,NaN
005 - LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT,NaN,16.0,NaN,17.0,106.0,12.0,NaN,NaN,NaN,69.0,...,NaN,36.0,46.0,NaN,13.0,NaN,13.0,16.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPAL DIAGNOSIS W CC,NaN,154.0,102.0,183.0,485.0,24.0,60.0,65.0,67.0,767.0,...,21.0,174.0,395.0,19.0,217.0,11.0,69.0,79.0,59.0,NaN
983 - EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPAL DIAGNOSIS W/O CC/MCC,NaN,11.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN
987 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL DIAGNOSIS W MCC,NaN,37.0,26.0,NaN,25.0,NaN,17.0,21.0,20.0,73.0,...,NaN,33.0,79.0,NaN,NaN,NaN,NaN,35.0,25.0,NaN


### Aside: lets quick check out what are the most frequent surgeries

In [16]:
# for our map, we are going to want to 
# normalize the discharges or each surgery 
# for each 
# state by the total discharges across all 
# states for a particular type of surger
#  lets add this to our total_discharges DF
total_discharges["Total"] = total_discharges.sum(axis = 1)
total_discharges["Total"].head() # just look at the first few

DRG Definition
001 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W MCC                1495.0
002 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W/O MCC                23.0
003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W MAJ O.R.    11987.0
004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W/O MAJ O.R.          11774.0
005 - LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT                          741.0
Name: Total, dtype: float64

In [17]:
# finally, lets check out the most often
#  performed surgery across all states

# we can do this by sorting our DF by this total we just
# calculated:
total_discharges.sort_values(by = "Total", 
                             ascending=False, 
                             inplace = True)

# now lets just look at the first few of our 
#  sorted array
total_discharges.head()

# so, from this we see that joint replacement
#  or reattachment of a lower extremeity is 
#  the most likely surgery (in number of discharges)
# followed by surgeries for sepsis and then heart failure

Provider State,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,TN,TX,UT,VA,VT,WA,WI,WV,WY,Total
DRG Definition,,,,,,,,,,,,,,,,,,,,,
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC,855.0,9146.0,5692.0,10472.0,32930.0,7793.0,5689.0,1247.0,2119.0,31086.0,...,11133.0,29410.0,4455.0,13923.0,766.0,10027.0,9482.0,2925.0,651.0,458259.0
871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC,452.0,6066.0,5187.0,6914.0,45434.0,4400.0,6390.0,643.0,1556.0,25209.0,...,11295.0,30595.0,2780.0,15119.0,739.0,10146.0,6712.0,3427.0,474.0,438597.0
291 - HEART FAILURE & SHOCK W MCC,110.0,3894.0,2143.0,2252.0,14894.0,1273.0,2806.0,529.0,1019.0,14923.0,...,5285.0,13718.0,500.0,6815.0,303.0,3557.0,3028.0,1535.0,161.0,198889.0
292 - HEART FAILURE & SHOCK W CC,166.0,4131.0,2382.0,2092.0,11654.0,1341.0,2714.0,902.0,936.0,14274.0,...,4899.0,13047.0,548.0,5863.0,318.0,2900.0,3175.0,1749.0,212.0,192918.0
"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC",182.0,4920.0,2462.0,3050.0,12358.0,1728.0,2723.0,556.0,782.0,20322.0,...,4681.0,12692.0,492.0,4884.0,249.0,2435.0,2582.0,1632.0,168.0,190375.0


In [18]:
# neat.  We won't need these for plotting, so we can remove our
# total column we just calculated
del total_discharges["Total"]
total_discharges.head()
# now we see that we are back to just states & surgeries
#  *but* our sorting is still by the total that we 
# previously calculated.
# spiffy!

Provider State,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
DRG Definition,,,,,,,,,,,,,,,,,,,,,
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC,855.0,9146.0,5692.0,10472.0,32930.0,7793.0,5689.0,1247.0,2119.0,31086.0,...,3200.0,11133.0,29410.0,4455.0,13923.0,766.0,10027.0,9482.0,2925.0,651.0
871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC,452.0,6066.0,5187.0,6914.0,45434.0,4400.0,6390.0,643.0,1556.0,25209.0,...,1574.0,11295.0,30595.0,2780.0,15119.0,739.0,10146.0,6712.0,3427.0,474.0
291 - HEART FAILURE & SHOCK W MCC,110.0,3894.0,2143.0,2252.0,14894.0,1273.0,2806.0,529.0,1019.0,14923.0,...,540.0,5285.0,13718.0,500.0,6815.0,303.0,3557.0,3028.0,1535.0,161.0
292 - HEART FAILURE & SHOCK W CC,166.0,4131.0,2382.0,2092.0,11654.0,1341.0,2714.0,902.0,936.0,14274.0,...,511.0,4899.0,13047.0,548.0,5863.0,318.0,2900.0,3175.0,1749.0,212.0
"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC",182.0,4920.0,2462.0,3050.0,12358.0,1728.0,2723.0,556.0,782.0,20322.0,...,436.0,4681.0,12692.0,492.0,4884.0,249.0,2435.0,2582.0,1632.0,168.0


## 1.c: plot data with bqplot

In [19]:
import bqplot
# by default bqplot does not import 
#  all packages, we have to 
#  explicitely import market_map
import bqplot.market_map # for access to market_map

In [20]:
# lets do our usual thing, but with a market map
#  instead of a heat map

# scales:
x_sc, y_sc = bqplot.OrdinalScale(), bqplot.OrdinalScale() # note, just a different way to call things
c_sc = bqplot.ColorScale(scheme="Blues")

# just a color axes for now:
c_ax = bqplot.ColorAxis(scale = c_sc, orientation = 'vertical')

# lets make the market map:

# (1) what should we plot for our color? lets take a look:
total_discharges.iloc[0].values, total_discharges.columns.values
# this is the total discharges for the most 
# popular surgical procedure
# the columns will be states

# (2) lets put this into a map
mmap = bqplot.market_map.MarketMap(color = total_discharges.iloc[0].values, 
                                  names = total_discharges.columns.values,
                                  scales={'color':c_sc}, 
                                   axes=[c_ax])

# (3) ok, but just clicking on things doesn't tell us too much
# lets add a little label to print out the total of the selected
import ipywidgets
label = ipywidgets.Label()
# link to market map
def get_data(change):
    # (3.1)
    #print(change['owner'].selected)
    # (3.2) loop
    v = 0.0 # to store total value
    for s in change['owner'].selected:
        v += total_discharges.iloc[0][total_discharges.iloc[0].index == s].values
    if v > 0: # in case nothing is selected
        # what are we printing?
        l = 'Total discharges of ' + \
        total_discharges.iloc[0].name + \
        ' = ' + str(v[0]) # note: v is by default an array
        label.value = l 
    
mmap.observe(get_data,'selected')
    
#mmap

# (3)
ipywidgets.VBox([label,mmap])

## Discussion:
* think back to the map we had last week: we can certainly plot this information with a more geo-realistic map
* what are the pros & cons of each style of map?  What do each highlight?  How are each biased?

## IF we have time: Re-do with other mapping system:

In [21]:
from us_state_abbrev import us_state_abbrev

sc_geo = bqplot.AlbersUSA()
state_data = bqplot.topo_load('map_data/USStatesMap.json')

#(1)
#states_map = bqplot.Map(map_data=state_data, scales={'projection':sc_geo})

#(2)
# library from last time
from states_utils import get_ids_and_names
ids, state_names = get_ids_and_names(states_map)

# color maps
import matplotlib.cm as cm
cmap = cm.Blues

# most popular surgery
popSurg = total_discharges.iloc[0]

# here, we will go through the process of getting colors to plot
#  each state with its similar color to the marketmap above:

#!pip install webcolors
from webcolors import rgb_to_hex
d = {} # empty dict to store colors
for s in states_map.map_data['objects']['subunits']['geometries']:
    if s['properties'] is not None:
        #print(s['properties']['name'], s['id'])
        # match states to abbreviations
        state_abbrev = us_state_abbrev[s['properties']['name']]
        #print(state_abbrev)
        v = popSurg[popSurg.index == state_abbrev].values[0]
        # renorm v to colors and then number of states
        v = (v - popSurg.values.min())/(popSurg.values.max()-popSurg.values.min())
        #print(v, int(cmap(v)[0]), int(cmap(v)[1]), int(cmap(v)[2]))
        # convert to from 0-1 to 0-255 rgbs
        c = [int(cmap(v)[i]*255) for i in range(3)]
        #d[s['id']] = rgb_to_hex([int(cmap(v)[0]*255), int(cmap(v)[1]*255), int(cmap(v)[2]*255)])
        d[s['id']] = rgb_to_hex(c)
    
    
def_tt = bqplot.Tooltip(fields=['name'])
    
states_map = bqplot.Map(map_data=state_data, scales={'projection':sc_geo}, colors = d, tooltip=def_tt)
# add interactions
states_map.interactions = {'click': 'select', 'hover': 'tooltip'}

# (3)
label = ipywidgets.Label()
# link to heat map
def get_data(change):
    v = 0.0 # to store total value
    if change['owner'].selected is not None:
        for s in change['owner'].selected:
            #print(s)
            sn = state_names[s == ids][0]
            state_abbrev = us_state_abbrev[sn]
            v += popSurg[popSurg.index == state_abbrev].values[0]
        if v > 0: # in case nothing is selected
            # what are we printing?
            l = 'Total discharges of ' + \
            popSurg.name + \
            ' = ' + str(v) # note: v is by default an array
            label.value = l 
    
states_map.observe(get_data,'selected')

fig=bqplot.Figure(marks=[states_map], 
                  title='US States Map Example',
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0}) # try w/o first and see
#fig
# (3)
ipywidgets.VBox([label,fig])

# Activity #2: Real quick ipyleaflets
* since cartopy wasn't working for folks, we'll quickly look at another option: ipyleaflets

In [22]:
#!pip install ipyleaflet
from ipyleaflet import *
# note: you might have to close and reopen you notebook
# to see the map

m = Map(center=(52, 10), zoom=8, basemap=basemaps.Hydda.Full)

#(2) street maps
strata_all = basemap_to_tiles(basemaps.Strava.All)
m.add_layer(strata_all)
m

Map(center=[52, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

### Note: more examples available here - https://github.com/jupyter-widgets/ipyleaflet/tree/master/examples